In [1]:
import pyrust_poker # pyrust_poker
import numpy as np

In [ ]:
len(pyrust_poker.poker_hand.get_cards())

poker_keys = np.array([card.key for card in pyrust_poker.poker_hand.get_cards()], dtype=np.uint64)
poker_mask = np.array([card.mask for card in pyrust_poker.poker_hand.get_cards()], dtype=np.uint64)

print(poker_keys.dtype, poker_mask.dtype)
print(poker_keys.shape, poker_mask.shape)

In [3]:
poker_keys = np.array([card.key for card in pyrust_poker.poker_hand.get_cards()], dtype=np.uint64)
poker_mask = np.array([card.mask for card in pyrust_poker.poker_hand.get_cards()], dtype=np.uint64)


def get_hand_strength(hand_masks: np.array):
  keep_keys = np.multiply(poker_keys, hand_masks).astype(np.uint64)
  keep_mask = np.multiply(poker_mask, hand_masks).astype(np.uint64)
  
  cards_sum = np.sum(keep_keys)
  cards_mask = np.bitwise_or.reduce(keep_mask)
  
  hand = pyrust_poker.poker_hand.Hand()
  hand.key += cards_sum
  hand.mask += cards_mask
  
  return pyrust_poker.evaluate(hand)
  

In [4]:
def get_hand_strength_matrix(hand_masks: np.array):
  keep_keys = np.multiply(poker_keys, hand_masks).astype(np.uint64)
  keep_mask = np.multiply(poker_mask, hand_masks).astype(np.uint64)
  
  hand = pyrust_poker.poker_hand.Hand()
  
  all_cards_sum = hand.key + np.sum(keep_keys, axis=1)
  all_cards_mask = np.bitwise_or.reduce(keep_mask, axis=1)
  
  
  results = np.zeros(shape=len(hand_masks))
  
  for index, (all_cards_sum, all_cards_mask) in enumerate(zip(all_cards_sum, all_cards_mask)):
    hand.key = all_cards_sum
    hand.mask = all_cards_mask
    
    results[index] = pyrust_poker.evaluate(hand)
      
  return results

In [ ]:
keep_cards_mask = np.zeros(shape=(2, 52), dtype=np.uint64)

keep_cards_mask[0, [0, 4, 8, 12, 48]] = 1 # 36865
keep_cards_mask[1, [48, 44, 40, 36, 32]] = 1 # 36874

get_hand_strength_matrix(keep_cards_mask)

In [ ]:
import time

hands_in_matrix = 10_000

keep_cards_mask = np.zeros(shape=(hands_in_matrix, 52), dtype=np.uint64)
keep_cards_mask[:, [0, 4, 8, 12, 48]] = 1

iterations = 100  # Number of evaluations to test
start_time = time.time()

for _ in range(iterations):
    get_hand_strength_matrix(keep_cards_mask)

end_time = time.time()
elapsed_time = end_time - start_time
evaluations_per_second = hands_in_matrix * iterations / elapsed_time

print("evaluations_per_second", evaluations_per_second)

In [7]:
def get_hand_strength_matrix(hand_masks: np.array):
  keep_keys = np.multiply(poker_keys, hand_masks).astype(np.uint64)
  keep_mask = np.multiply(poker_mask, hand_masks).astype(np.uint64)
  
  hand = pyrust_poker.poker_hand.Hand()
  
  all_cards_sum = hand.key + np.sum(keep_keys, axis=1)
  all_cards_mask = np.bitwise_or.reduce(keep_mask, axis=1)
  
  eval_list = list(zip(all_cards_sum, all_cards_mask))
  
  return pyrust_poker.evaluate_list(eval_list)

In [ ]:
get_hand_strength_matrix(keep_cards_mask[:3])

In [ ]:
import time

hands_in_matrix = 10_000

keep_cards_mask = np.zeros(shape=(hands_in_matrix, 52), dtype=np.uint64)
keep_cards_mask[:, [0, 4, 8, 12, 48]] = 1

iterations = 1_000  # Number of evaluations to test
start_time = time.time()



for _ in range(iterations):
    get_hand_strength_matrix(keep_cards_mask)

end_time = time.time()
elapsed_time = end_time - start_time
evaluations_per_second = hands_in_matrix * iterations / elapsed_time

print("evaluations_per_second", evaluations_per_second)